In [1]:
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm_notebook as tqdm

In [2]:
with open('/Users/zhiyulin/Downloads/oral_arguments/data/transcripts/2019/63106.json') as f:
    data = json.load(f)

In [4]:
#https://stackoverflow.com/questions/52081545/python-3-flattening-nested-dictionaries-and-lists-within-dictionaries

def flatten(input_dict, separator='_', prefix=''):
    output_dict = {}
    for key, value in input_dict.items():
        if isinstance(value, dict) and value:
            deeper = flatten(value, separator, prefix+key+separator)
            output_dict.update({key2: val2 for key2, val2 in deeper.items()})
        elif isinstance(value, list) and value:
            for index, sublist in enumerate(value, start=1):
                if isinstance(sublist, dict) and sublist:
                    deeper = flatten(sublist, separator, prefix+key+separator+str(index)+separator)
                    output_dict.update({key2: val2 for key2, val2 in deeper.items()})
                else:
                    output_dict[prefix+key+separator+str(index)] = value
        else:
            output_dict[prefix+key] = value
    return output_dict

def create_df(data):
    df = pd.DataFrame()
    for j in range(1, len(data['transcript']['sections'])+1):
        for i in range(1, len(data['transcript']['sections'][j-1]['turns'])+1):
            try:
                temp = data['transcript']['sections'][j-1]['turns'][i-1]
                row = pd.DataFrame(flatten(temp), index=[i*j])
                df = pd.concat([df, row], axis=0)
            except Exception as e:
                print(str(e))
                pass
    return df

exp_df = create_df(data)
exp_df

,start,stop,byte_start,byte_stop,speaker_ID,speaker_name,speaker_href,speaker_view_count,speaker_last_name,speaker_roles_1_id,...,text_blocks_22_start,text_blocks_22_stop,text_blocks_22_byte_start,text_blocks_22_byte_stop,text_blocks_22_text,text_blocks_23_start,text_blocks_23_stop,text_blocks_23_byte_start,text_blocks_23_byte_stop,text_blocks_23_text
1,0.00,8.92,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.92,166.28,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,166.28,167.40,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167.40,168.76,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,168.76,184.24,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,3379.88,3383.20,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3383.20,3646.16,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,3631.36,3635.32,0.0,0.0,So in a sense Apodaca itself was born of a dis...,3635.32,3646.16,0.0,0.0,And so if you feel strongly about stare decisi...
6,3646.16,3686.00,0,0,15068,"Samuel A. Alito, Jr.",https://api.oyez.org/people/samuel_a_alito_jr,0,Alito,2712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3686.00,3705.00,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
actions = exp_df[exp_df['speaker_roles_1_type'] == 'scotus_justice']
states = exp_df[exp_df['speaker_roles_1_type'] != 'scotus_justice']
pairs = states.merge(actions, left_on='stop', right_on='start')
states

,start,stop,byte_start,byte_stop,speaker_ID,speaker_name,speaker_href,speaker_view_count,speaker_last_name,speaker_roles_1_id,...,text_blocks_22_start,text_blocks_22_stop,text_blocks_22_byte_start,text_blocks_22_byte_stop,text_blocks_22_text,text_blocks_23_start,text_blocks_23_stop,text_blocks_23_byte_start,text_blocks_23_byte_stop,text_blocks_23_text
2,8.92,166.28,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167.40,168.76,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,184.24,254.32,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,316.68,348.56,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,349.56,351.72,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,3346.36,3362.68,0,0,62953,Elizabeth Murrill,https://api.oyez.org/people/elizabeth_murrill,0,Murrill,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,3364.56,3365.68,0,0,62953,Elizabeth Murrill,https://api.oyez.org/people/elizabeth_murrill,0,Murrill,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,3375.88,3379.88,0,0,62953,Elizabeth Murrill,https://api.oyez.org/people/elizabeth_murrill,0,Murrill,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3383.20,3646.16,0,0,30312,Jeffrey Fisher,https://api.oyez.org/people/jeffrey_fisher,0,Fisher,NaN,...,3631.36,3635.32,0.0,0.0,So in a sense Apodaca itself was born of a dis...,3635.32,3646.16,0.0,0.0,And so if you feel strongly about stare decisi...


In [23]:
actions

,start,stop,byte_start,byte_stop,speaker_ID,speaker_name,speaker_href,speaker_view_count,speaker_last_name,speaker_roles_1_id,...,text_blocks_22_start,text_blocks_22_stop,text_blocks_22_byte_start,text_blocks_22_byte_stop,text_blocks_22_text,text_blocks_23_start,text_blocks_23_stop,text_blocks_23_byte_start,text_blocks_23_byte_stop,text_blocks_23_text
1,0.00,8.92,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,166.28,167.40,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,168.76,184.24,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,254.32,316.68,0,0,15068,"Samuel A. Alito, Jr.",https://api.oyez.org/people/samuel_a_alito_jr,0,Alito,2712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,348.56,349.56,0,0,15068,"Samuel A. Alito, Jr.",https://api.oyez.org/people/samuel_a_alito_jr,0,Alito,2712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,3362.68,3364.56,0,0,15084,Ruth Bader Ginsburg,https://api.oyez.org/people/ruth_bader_ginsburg,0,Ginsburg,2728.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,3365.68,3375.88,0,0,15084,Ruth Bader Ginsburg,https://api.oyez.org/people/ruth_bader_ginsburg,0,Ginsburg,2728.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,3379.88,3383.20,0,0,15086,"John G. Roberts, Jr.",https://api.oyez.org/people/john_g_roberts_jr,0,Roberts,2730.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3646.16,3686.00,0,0,15068,"Samuel A. Alito, Jr.",https://api.oyez.org/people/samuel_a_alito_jr,0,Alito,2712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pairs.columns.tolist()

['start_x',
 'stop_x',
 'byte_start_x',
 'byte_stop_x',
 'speaker_ID_x',
 'speaker_name_x',
 'speaker_href_x',
 'speaker_view_count_x',
 'speaker_last_name_x',
 'speaker_roles_1_id_x',
 'speaker_roles_1_type_x',
 'speaker_roles_1_date_start_x',
 'speaker_roles_1_date_end_x',
 'speaker_roles_1_appointing_president_x',
 'speaker_roles_1_role_title_x',
 'speaker_roles_1_institution_name_x',
 'speaker_roles_1_href_x',
 'speaker_thumbnail_id_x',
 'speaker_thumbnail_mime_x',
 'speaker_thumbnail_size_x',
 'speaker_thumbnail_href_x',
 'speaker_length_of_service_x',
 'speaker_identifier_x',
 'text_blocks_1_start_x',
 'text_blocks_1_stop_x',
 'text_blocks_1_byte_start_x',
 'text_blocks_1_byte_stop_x',
 'text_blocks_1_text_x',
 'speaker_roles_x',
 'speaker_thumbnail_x',
 'text_blocks_2_start_x',
 'text_blocks_2_stop_x',
 'text_blocks_2_byte_start_x',
 'text_blocks_2_byte_stop_x',
 'text_blocks_2_text_x',
 'text_blocks_3_start_x',
 'text_blocks_3_stop_x',
 'text_blocks_3_byte_start_x',
 'text_bloc

In [19]:
pairs['speaker_last_name_y']

0       Roberts
1       Roberts
2         Alito
3         Alito
4         Alito
         ...   
104    Ginsburg
105    Ginsburg
106     Roberts
107       Alito
108     Roberts
Name: speaker_last_name_y, Length: 109, dtype: object

In [21]:
texts = [e for e in pairs.columns.tolist() if ('text_blocks' in e) & ('_text' in e)]
pairs[texts]

,text_blocks_1_text_x,text_blocks_2_text_x,text_blocks_3_text_x,text_blocks_4_text_x,text_blocks_5_text_x,text_blocks_6_text_x,text_blocks_7_text_x,text_blocks_8_text_x,text_blocks_9_text_x,text_blocks_10_text_x,...,text_blocks_14_text_y,text_blocks_15_text_y,text_blocks_16_text_y,text_blocks_17_text_y,text_blocks_18_text_y,text_blocks_19_text_y,text_blocks_20_text_y,text_blocks_21_text_y,text_blocks_22_text_y,text_blocks_23_text_y
0,"Mr. Chief Justice, and may it please the Court...","And, indeed, that reasoning flouted precedent ...",As the Court has said many times over many dec...,"In particular, what the Court has said is that...",Unanimity is an absolute requirement to trial ...,And the reasons that the common law commentato...,"In a nutshell, we are not prepared to take awa...","Instead, what the state says are two primary t...",And we think for three reasons the state has o...,The state talks about the fact that many state...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,And I think --,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Well, the best historical evidence, Mr. Chief ...",And so what James Madison did is take away all...,And then Justice Wilson actually amended the -...,"And, remember, Justice Wilson, as we note at l...",But what you have is the reverse.,"You have Justice Wilson, right after the Const...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Well, I think Justice Alito, I'd like to make ...","Instead, they're asking the Court to adopt a n...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,But here the state is asking for a brand-new r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,"Justice Ginsburg, I -- I think that it was not...",I would disagree with that -- that description...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,-- been married together.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,I think that characterization of it was dismis...,That's all.,Thank you.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,Thank you.,I'd like to make a couple of quick factual poi...,"So we say in our reply brief, using one of the...","And so we think, at least in the direct review...",And so that would be a question about the Insu...,You're going to be talking about that next wee...,One is the Sixth Amendment requires unanimous ...,It would raise questions like the one the Chie...,And that would just be one of many questions t...,I think it's also telling that Oregon is not w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
sub = pairs[['start_x', 'stop_x', 'start_y', 'stop_y', 'speaker_name_x', 'speaker_name_y', 
             'speaker_roles_1_appointing_president_y', 
             'speaker_roles_1_role_title_y',
             'speaker_roles_1_date_start_y',
             'speaker_roles_1_date_end_y'] + texts]
sub

,start_x,stop_x,start_y,stop_y,speaker_name_x,speaker_name_y,speaker_roles_1_appointing_president_y,speaker_roles_1_role_title_y,speaker_roles_1_date_start_y,speaker_roles_1_date_end_y,...,text_blocks_14_text_y,text_blocks_15_text_y,text_blocks_16_text_y,text_blocks_17_text_y,text_blocks_18_text_y,text_blocks_19_text_y,text_blocks_20_text_y,text_blocks_21_text_y,text_blocks_22_text_y,text_blocks_23_text_y
0,8.92,166.28,166.28,167.40,Jeffrey Fisher,"John G. Roberts, Jr.",George W. Bush,Chief Justice of the United States,1.127970e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167.40,168.76,168.76,184.24,Jeffrey Fisher,"John G. Roberts, Jr.",George W. Bush,Chief Justice of the United States,1.127970e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,184.24,254.32,254.32,316.68,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.138687e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,316.68,348.56,348.56,349.56,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.138687e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,349.56,351.72,351.72,371.52,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.138687e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3346.36,3362.68,3362.68,3364.56,Elizabeth Murrill,Ruth Bader Ginsburg,Bill Clinton,Associate Justice of the Supreme Court of the ...,7.449588e+08,1.600405e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,3364.56,3365.68,3365.68,3375.88,Elizabeth Murrill,Ruth Bader Ginsburg,Bill Clinton,Associate Justice of the Supreme Court of the ...,7.449588e+08,1.600405e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,3375.88,3379.88,3379.88,3383.20,Elizabeth Murrill,"John G. Roberts, Jr.",George W. Bush,Chief Justice of the United States,1.127970e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,3383.20,3646.16,3646.16,3686.00,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.138687e+09,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
pd.DataFrame(sub.iloc[0][['start_x', 'stop_x', 'start_y', 'stop_y', 'speaker_name_x', 'speaker_name_y', 'agent_state', 'judge_action']]).T

,start_x,stop_x,start_y,stop_y,speaker_name_x,speaker_name_y,agent_state,judge_action
0,8.92,166.28,166.28,167.4,Jeffrey Fisher,"John G. Roberts, Jr.","Mr. Chief Justice, and may it please the Court: Last term in Timbs against Indiana, this Court reaffirmed the well-settled rule that incorporated provisions of the Bill of Rights apply the same way to the states as they apply to the federal government. Taking that rule as the given, the state does not defend Justice Powell's pivotal vote in the Apodaca case. And, indeed, that reasoning flouted precedent at the time and has since been relegated to nothing more than an isolated relic of an abandoned doctrine. The state's only defense in -- in support of the judgment below is that the Sixth Amendment does not require unanimity at all; that is, not in state courts or in federal courts. This Court should reject that argument. As the Court has said many times over many decades, the Sixth Amendment requires a unanimous verdict to convict. In particular, what the Court has said is that the Sixth Amendment right to trial by jury carries with it the essentials of the common law. And the common law authorities are uniform, explicit, and absolute. Unanimity is an absolute requirement to trial by jury. And the reasons that the common law commentators gave for that rule are the -- are -- resonate just as powerfully now as they did then. In a nutshell, we are not prepared to take away someone's liberty unless a cross-section of the community uniformly agrees that criminal punishment is appropriate. Now, I don't think the state disputes that historical account that I just gave you or even that unanimity is central to the proper functioning of the jury trial right. Instead, what the state says are two primary things: First, that the drafting history of the Sixth Amendment suggests that the framers meant to dispense with that historical rule, and, second, that that historical requirement of unanimity is no more important than the 12-person rule, which this Court said is not part of the Sixth Amendment, in Williams. So let me turn to those two arguments. Let me start with the drafting history. And we think for three reasons the state has over-read the drafting history. First, as the Court itself said in cases dealing with provisions like the Second Amendment and the Double Jeopardy Clause, we do not read into a deletion of language any meaning when there's no contemporary evidence that it was designed to change the meaning of the provision. And that's all the more true here because of the contextual backdrop. The state talks about the fact that many states at the time had trial by jury provisions in their own constitutions and correctly notes that some of those provisions explicitly required unanimity but some of them didn't. And the rule was the same across all of those states, so the thing that the framers would have taken from the context at the time would have been that it doesn't matter whether you have unanimity in the provision; it requires it either way.","Well, but still that --"


In [25]:
sub.columns

Index(['start_x', 'stop_x', 'start_y', 'stop_y', 'speaker_name_x',
       'speaker_name_y', 'speaker_roles_1_appointing_president_y',
       'speaker_roles_1_role_title_y', 'speaker_roles_1_date_start_y',
       'speaker_roles_1_date_end_y', 'text_blocks_1_text_x',
       'text_blocks_2_text_x', 'text_blocks_3_text_x', 'text_blocks_4_text_x',
       'text_blocks_5_text_x', 'text_blocks_6_text_x', 'text_blocks_7_text_x',
       'text_blocks_8_text_x', 'text_blocks_9_text_x', 'text_blocks_10_text_x',
       'text_blocks_11_text_x', 'text_blocks_12_text_x',
       'text_blocks_13_text_x', 'text_blocks_14_text_x',
       'text_blocks_15_text_x', 'text_blocks_16_text_x',
       'text_blocks_17_text_x', 'text_blocks_18_text_x',
       'text_blocks_19_text_x', 'text_blocks_20_text_x',
       'text_blocks_21_text_x', 'text_blocks_22_text_x',
       'text_blocks_23_text_x', 'text_blocks_1_text_y', 'text_blocks_2_text_y',
       'text_blocks_3_text_y', 'text_blocks_4_text_y', 'text_blocks_5_text

In [96]:
pd.DataFrame([start_x, stop_x, start_y, stop_y, speaker_name_x, speaker_name_y, this_state, this_action]).T

,0,1,2,3,4,5,6,7
0,3346.36,3365.68,3365.68,3375.88,Elizabeth Murrill,Ruth Bader Ginsburg,"I think I would -- I would say that would be different for all the reasons I just outlined. The Insular Cases were majority decisions from the Court. They were -- they were based on a view that has not been disregarded or left behind in the Court's jurisprudence. There may be arguments parties can make under ordinary stare decisis principles, but the last point I would leave you with is this is not an ordinary stare decisis case.","Thank you, counsel. The case is submitted."


In [26]:
sub.fillna('', inplace=True)
sub['state_dur'] = sub['stop_x'] - sub['start_x']
sub['action_dur'] = sub['stop_y'] - sub['start_y']
sub['agent_state'] = sub[[e for e in texts if 'text_x' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
sub['judge_action'] = sub[[e for e in texts if 'text_y' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
full_pairs = pd.DataFrame()

for i in range(sub.shape[0]):
    
    print('\ni beginning:',i)
    
    try:
        # if the attorney said something substantial (state)
        if  sub.iloc[i]['state_dur'] >= 2:

            # if the judge had a proper response (action)
            if sub.iloc[i]['action_dur'] >= 2:

                # continue to the next state
                to_append = pd.DataFrame(
                    sub.iloc[i][['start_x', 'stop_x', 'start_y', 'stop_y', 
                                 'speaker_name_x', 'speaker_name_y', 
                                 'speaker_roles_1_appointing_president_y', 
                                 'speaker_roles_1_role_title_y',
                                 'speaker_roles_1_date_start_y',
                                 'speaker_roles_1_date_end_y',
                                 'agent_state', 'judge_action']]).T
                print('to_append first:', to_append.shape)
                full_pairs = full_pairs.append(to_append)
                continue

            # if the judge didn't have a chance to give a proper response
            else:

                # remember the state (what the attorney said before), go the next rows until a proper response
                # is given (long enough action)
                this_state = sub.iloc[i]['agent_state']
                j = i
                this_action = ''
                while sub.iloc[j]['action_dur'] < 2:
                    this_action = this_action + sub.iloc[j]['judge_action']
                    j = j+1
                this_action = this_action + sub.iloc[j]['judge_action']
                speaker_name_x = sub.iloc[i]['speaker_name_x']
                speaker_name_y = sub.iloc[j]['speaker_name_y']
                start_x = sub.iloc[i]['start_x']
                stop_x = sub.iloc[j]['start_y']
                start_y = sub.iloc[j]['start_y']  
                stop_y = sub.iloc[j]['stop_y']  
                to_append = pd.DataFrame([start_x, stop_x, start_y, stop_y, speaker_name_x, speaker_name_y, this_state, this_action]).T
                to_append.columns = ['start_x', 'stop_x', 'start_y', 'stop_y', 
                                     'speaker_name_x', 'speaker_name_y', 
                                     'speaker_roles_1_appointing_president_y', 
                                     'speaker_roles_1_role_title_y',
                                     'speaker_roles_1_date_start_y',
                                     'speaker_roles_1_date_end_y',
                                     'agent_state', 'judge_action']
                print('to_append here:', to_append.shape)
                full_pairs = full_pairs.append(to_append)

                # go to the next full state
                skip_until = j+1  


        else:
            if skip_until >= i:
                continue   
            else:
                if sub.iloc[i]['judge_action'].startswith("--"):
                    full_pairs.iloc[-1]['judge_action'] = this_action + sub.iloc[i]['judge_action']
                else:
                    print('i is:', i, 'skip_until is:', skip_until)
                    print('something went wrong')
    except Exception as e:
        print(str(e))
        continue

        


i beginning: 0
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 1
name 'skip_until' is not defined

i beginning: 2
to_append first: (1, 12)

i beginning: 3
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 4
to_append first: (1, 12)

i beginning: 5
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 6
name 'skip_until' is not defined

i beginning: 7
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 8
to_append first: (1, 12)

i beginning: 9
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 10
name 'skip_until' is not defined

i beginning: 11
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 12
Length mismatch: Expected axis has 8 elements, new values have 12 elements

i beginning: 13
name 'skip_until' is not defined

i beginning: 14
Length mismatch: Expect

In [27]:
full_pairs

,start_x,stop_x,start_y,stop_y,speaker_name_x,speaker_name_y,speaker_roles_1_appointing_president_y,speaker_roles_1_role_title_y,speaker_roles_1_date_start_y,speaker_roles_1_date_end_y,agent_state,judge_action
2,184.24,254.32,254.32,316.68,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.13869e+09,0,"Well, the best historical evidence, Mr. Chief ...","You are asking us to overrule Apodaca, so we d..."
4,349.56,351.72,351.72,371.52,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.13869e+09,0,But here the state is asking for a brand-new r...,I -- I don't want to interrupt. That's a fair ...
8,430.64,435.52,435.52,477.12,Jeffrey Fisher,Elena Kagan,Barack Obama,Associate Justice of the Supreme Court of the ...,1.28116e+09,0,-- opinions from this Court that say the same ...,"I mean, it would be an outlier. It would be so..."
21,753.04,816.88,816.88,835.96,Jeffrey Fisher,Neil Gorsuch,Donald J. Trump,Associate Justice of the Supreme Court of the ...,1.49154e+09,0,"Well, Mr. Chief Justice, can I give you a lega...","Mr. Fisher, let's say I am not entirely persua..."
22,835.96,925.76,925.76,943.8,Jeffrey Fisher,Elena Kagan,Barack Obama,Associate Justice of the Supreme Court of the ...,1.28116e+09,0,"I think what I would say to you, Justice Gorsu...","Do you think, Mr. Fisher, that we would also h..."
24,1000.92,1049.52,1049.52,1078.24,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.13869e+09,0,"Well, I can tell you what I think and I can te...","Well, what about a party that has to make deci..."
25,1078.24,1122.36,1122.36,1171.88,Jeffrey Fisher,"Samuel A. Alito, Jr.",George W. Bush,Associate Justice of the Supreme Court of the ...,1.13869e+09,0,"Well, Justice Alito, I think that at least in ...",Can I come back to the -- the math question th...
28,1214.84,1256.52,1256.52,1262.32,Jeffrey Fisher,Brett M. Kavanaugh,Donald J. Trump,Associate Justice of the Supreme Court of the ...,1.5388e+09,0,"Justice Alito, perhaps there'd be a number whe...",Do the racial origins of this rule have an imp...
29,1262.32,1264.44,1264.44,1267.36,Jeffrey Fisher,Brett M. Kavanaugh,Donald J. Trump,Associate Justice of the Supreme Court of the ...,1.5388e+09,0,"I think they do, Justice Kavanaugh. I think --...",How? How do -- how should we factor those in? ...
32,1352.92,1411.36,1411.36,1429.2,Jeffrey Fisher,Elena Kagan,Barack Obama,Associate Justice of the Supreme Court of the ...,1.28116e+09,0,"No, Justice -- Justice Alito. Let me make sure...",You -- you mentioned a couple of times earlier...


In [5]:
def parse_all_json(years):
    for y in years:
        mypath = join('data', 'transcripts', y)
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
        print(y, ':', len(onlyfiles))

        mydf = pd.DataFrame()
        for f in tqdm(onlyfiles):
            try:
                with open(join(mypath, f)) as myfile:
                    data = json.load(myfile)
            except Exception as e:
                print(str(e))
                continue
            mydf = pd.concat([create_df(data), mydf], axis=0)
        if not os.path.exists('parsed_data'):
            os.mkdir('parsed_data')
        mydf.to_csv(join('parsed_data', str(y) + '_df.csv'))

In [6]:
years = next(os.walk('./data/transcripts'))[1]
parse_all_json(years)

2017 : 63


<ipython-input-5-d7bb88c61740>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f in tqdm(onlyfiles):


KeyboardInterrupt: 

### Convert df into data ready for NN

In [7]:
import os
import pandas as pd
import nltk
import numpy as np
import pandas as pd
import regex
import string
import unicodedata
import html
from html.parser import HTMLParser

from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.sentiment.util import  mark_negation
from nltk.tokenize import TweetTokenizer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report as clsr
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.model_selection import train_test_split

import gensim
import gensim.downloader as gensim_downloader
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import itertools

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Provided by Prof. Linda Moreau through ANLY590 NLP class with modifications
class Preprocessor(BaseEstimator, TransformerMixin):


    def __init__(self, lower=True, strip=True, remove_punct=True, remove_num=True, remove_diacritics=True, unicode_form='NFC', remove_stop=False, stem_tokens=False, lemmatize_tokens=True, language='english'):
        
        self.lower      = lower
        self.strip      = strip
        self.remove_punct = remove_punct
        self.remove_num = remove_num
        self.remove_diacritics = remove_diacritics
        self.set_unicode_form(unicode_form)
     
        self.remove_stop = remove_stop
        self.stem_tokens = stem_tokens
        self.lemmatize_tokens = lemmatize_tokens    

        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.punct      = set(string.punctuation)

        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = nltk.stem.SnowballStemmer(language) 
        self.tokenizer = TweetTokenizer()

 
    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def inverse_transform(self, X):
        return [" ".join(doc) for doc in X]


    def fit(self, X, y=None):
        return self

 
    def set_stopwords(self, stopwords):
        self.stopwords  = stopwords or set(nltk.corpus.stopwords.words('english'))


    def set_punct(self, punct):
        self.punct      = punct or set(string.punctuation)


    def set_unicode_form(self, unicode_form):
        self.unicode_form = unicode_form.strip().upper()
        if not self.unicode_form in ['NFC','NFD','NFKC','NFKD']:
            self.unicode_form = 'NFC'


    def show_settings(self):
        print("lower: ",self.lower)
        print("strip: ",self.strip)
        print("remove_punct: ",self.remove_punct)
        print("remove_diacritics: ",self.remove_diacritics)
        print("unicode_form: ",self.unicode_form)
        print("remove_stop: ",self.remove_stop)
        print("stem_tokens: ",self.stem_tokens)
        print("lemmatize_tokens",self.lemmatize_tokens)       
        print("lemmatizer: ",type(self.lemmatizer).__name__)
        print("stemmer: ",type(self.stemmer).__name__) 
        print("punctuation: ",self.punct)
        print("stopwords: ",self.stopwords)


    # convert nltk pos-tag to wordnet pos-tag
    # for use by the wordnet lemmatizer
    @staticmethod
    def get_wordnet_tag(nltk_tag):
        if nltk_tag.startswith('N'):
            return wn.NOUN
        elif nltk_tag.startswith('V'):
            return wn.VERB
        elif nltk_tag.startswith('J'):
            return wn.ADJ
        elif nltk_tag.startswith('R'):
            return wn.ADV
        else:          
            return None

    # determine whether a string is comprised of all punctuation-like characters
    def is_punct(self, text):
        if text in self.punct:
            return True
        if regex.match(r"[\p{P}\p{Mn}\p{Sk}]+", text):
            return True
        return False
    
    # determine whether a string contains numbers
    def has_number(self, text):
        return any(char.isdigit() for char in text)


    # normalize token strings
    def normalize_string(self, token):

        token = html.unescape(token)
        token = token.lower() if self.lower else token
        token = token.strip(' _*')  if self.strip else token

        if self.remove_diacritics:
            token = regex.sub("\p{Mn}",'',unicodedata.normalize('NFD',token)) 

        #not is in the stopword list, but n't isn'
        if token == "n't" and self.stopwords:
            token = "not"
      
        return unicodedata.normalize(self.unicode_form,token)

    # tokenize the document with optional normalization
    def tokenize(self, document, all_fields=False):
        tokens = []
        for sent in sent_tokenize(document):
            cleaner_sent = html.unescape(sent)
            for token in nltk.pos_tag(self.tokenizer.tokenize(cleaner_sent)):
                stem = ''
                token_text = self.normalize_string(token[0])
                token_pos = token[1]
                if self.remove_punct and self.is_punct(token_text):
                    continue
                if self.remove_num and self.has_number(token_text):
                    continue
                if self.remove_stop and token_text in self.stopwords:
                    continue
                if self.stem_tokens or all_fields:
                    stem = self.stemmer.stem(token_text)
                if self.lemmatize_tokens or all_fields:
                    wordnet_tag = self.get_wordnet_tag(token_pos)
                    if wordnet_tag is not None:
                        lemma = self.lemmatizer.lemmatize(token_text,wordnet_tag)
                    else:
                        lemma = token_text
                if all_fields:
                    tokens.append({'token': token_text, 'stem': stem, 'lemma': lemma})
                elif self.stem_tokens:
                    tokens.append(stem)
                elif self.lemmatize_tokens:
                    tokens.append(lemma)
                else:
                    tokens.append(token_text)     
        return tokens

In [28]:
import warnings
warnings.filterwarnings("ignore")


class PrepareData():
    
    def __init__(self, input_directory, output_directory, model_name, min_duration, feature_size, context_window_size, min_words, down_sample):
        self.input_directory = input_directory
        self.output_directory = output_directory
        self.min_duration = min_duration
        self.feature_size = feature_size
        self.context_window_size = context_window_size
        self.min_words = min_words
        self.down_sample = down_sample
        self.model_name = model_name
        self.model_file = None
        
    def load_data(self):
        print("\nStep: Loading data...")
        DF = pd.DataFrame()
        onlyfiles = [f for f in listdir(self.input_directory) if isfile(join(self.input_directory, f))]
        for f in onlyfiles:
            df = pd.read_csv(join(self.input_directory, f), index_col=0)
            DF = pd.concat([DF, df], axis=0)
        print("Input dataframe shape:", DF.shape)
        return DF
    
    def infer_conversation_turns(self, sub):
        full_pairs = pd.DataFrame()
        for i in range(sub.shape[0]):
            try:
                # if the attorney said something substantial (state)
                if  sub.iloc[i]['state_dur'] >= 2:

                    # if the judge had a proper response (action)
                    if sub.iloc[i]['action_dur'] >= 2:

                        # continue to the next state
                        to_append = pd.DataFrame(
                            sub.iloc[i][['start_x', 'stop_x', 'start_y', 'stop_y', 
                                         'speaker_name_x', 'speaker_name_y', 
                                         'speaker_roles_1_appointing_president_y', 
                                         'speaker_roles_1_role_title_y',
                                         'speaker_roles_1_date_start_y',
                                         'speaker_roles_1_date_end_y',
                                         'agent_state', 'judge_action']]).T
                        full_pairs = full_pairs.append(to_append)
                        continue

                    # if the judge didn't have a chance to give a proper response
                    else:

                        # remember the state (what the attorney said before), go the next rows until a proper response
                        # is given (long enough action)
                        this_state = sub.iloc[i]['agent_state']
                        j = i
                        this_action = ''
                        while sub.iloc[j]['action_dur'] < 2:
                            this_action = this_action + sub.iloc[j]['judge_action']
                            j = j+1
                        this_action = this_action + sub.iloc[j]['judge_action']
                        speaker_name_x = sub.iloc[i]['speaker_name_x']
                        speaker_name_y = sub.iloc[j]['speaker_name_y']
                        start_x = sub.iloc[i]['start_x']
                        stop_x = sub.iloc[j]['start_y']
                        start_y = sub.iloc[j]['start_y']  
                        stop_y = sub.iloc[j]['stop_y']  
                        to_append = pd.DataFrame([start_x, stop_x, start_y, stop_y, speaker_name_x, speaker_name_y, this_state, this_action]).T
                        to_append.columns = ['start_x', 'stop_x', 'start_y', 'stop_y', 
                                             'speaker_name_x', 'speaker_name_y', 
                                             'speaker_roles_1_appointing_president_y', 
                                             'speaker_roles_1_role_title_y',
                                             'speaker_roles_1_date_start_y',
                                             'speaker_roles_1_date_end_y',
                                             'agent_state', 'judge_action']
                        full_pairs = full_pairs.append(to_append)

                        # go to the next full state
                        skip_until = j+1  


                else:
                    if skip_until >= i:
                        continue   
                    else:
                        if sub.iloc[i]['judge_action'].startswith("--"):
                            full_pairs.iloc[-1]['judge_action'] = this_action + sub.iloc[i]['judge_action']
                        else:
                            pass
            except Exception as e:
                continue
        return full_pairs

    def create_action_state_pair(self, df):
        print("\nStep: Creating action state pair...")
        # Pair state and action by start and stop time
        actions = df[df['speaker_roles_1_type'] == 'scotus_justice']
        states = df[df['speaker_roles_1_type'] != 'scotus_justice']
        pairs = states.merge(actions, left_on='stop', right_on='start')
        texts = [e for e in pairs.columns.tolist() if ('text_blocks' in e) & ('_text' in e)]

        # collapse action and state sentences into two columns
        sub = pairs[['start_x', 'stop_x', 'start_y', 'stop_y', 'speaker_name_x', 'speaker_name_y',
                    'speaker_roles_1_appointing_president_y', 
                     'speaker_roles_1_role_title_y',
                     'speaker_roles_1_date_start_y',
                     'speaker_roles_1_date_end_y',] + texts]
        sub.fillna('', inplace=True)
#         sub['agent_state'] = sub[[e for e in texts if 'text_x' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#         sub['judge_action'] = sub[[e for e in texts if 'text_y' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        sub['state_dur'] = sub['stop_x'] - sub['start_x']
        sub['action_dur'] = sub['stop_y'] - sub['start_y']
        sub['agent_state'] = sub[[e for e in texts if 'text_x' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        sub['judge_action'] = sub[[e for e in texts if 'text_y' in e]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        mysub = self.infer_conversation_turns(sub)
#         mysub = sub[['start_x', 'stop_x', 'start_y', 'stop_y', 'speaker_name_x', 'speaker_name_y', 'judge_action', 'agent_state']]

        # Only keep when the actions and states that last longer than n seconds
        mysub['action_duration'] = mysub['stop_x'] - mysub['start_x']
        mysub['agent_duration'] = mysub['stop_y'] - mysub['start_y']
        mysub = mysub[(mysub['action_duration'] > self.min_duration) & (mysub['agent_duration'] > self.min_duration)].reset_index(drop=True)
        return mysub
    
    def preprocess_text(self, df):
        print("\nStep: Preprocessing text...")
        my_preprocessor = Preprocessor(lower=True, strip=True, remove_punct=True, 
                                       remove_num=True, remove_diacritics=True, 
                                       remove_stop=True, stem_tokens=False, lemmatize_tokens=True)
        df['judge_action_cleaned'] = df['judge_action'].apply(my_preprocessor.tokenize)
        df['agent_state_cleaned'] = df['agent_state'].apply(my_preprocessor.tokenize)
        return df
        
    def train_w2v(self, action_sentences, state_sentences):
        print("\nStep: Training Word2Vec...")
        # download gensim resources
        gensim_resources = gensim_downloader.info(name_only=True)
        
        # load googlenews w2v model
        googlenews_w2v = "word2vec-google-news-300"
        gensim_resources = gensim_downloader.info(name=googlenews_w2v)
        googlenews_w2v_model = gensim_downloader.load(googlenews_w2v)
        
        print("Action and state length:", len(action_sentences))
        
        # train the model
        model = Word2Vec(action_sentences + state_sentences, 
                           size = self.feature_size, 
                           window = self.context_window_size,
                           min_count = self.min_words,
                           sample = self.down_sample)
        self._save_w2v(model)
        return model
        
    def _save_w2v(self, model):
        # save the model
        if not os.path.exists(self.output_directory):
            os.mkdir(self.output_directory)
        with open (join(self.output_directory, self.model_name + '.model'), "w") as model_file:
            model.save(model_file.name)
        del model
        self.model_file = model_file

    def load_w2v(self):
        model = gensim.models.KeyedVectors.load(self.model_file.name, mmap='r')
        print("Model vocabulary:", len(model.wv.vocab))
        return model
        
    def create_x_y(self, action_sentences, state_sentences, wv_model):
        print("\nStep: Creating input X and y...")
        A = []
        S = []
        for a, s in zip(action_sentences, state_sentences):
            A.append( np.array([wv_model.wv[word] for word in a]) )
            S.append( np.array([wv_model.wv[word] for word in s]) )
        padded_A = pad_sequences(A, padding='post')
        padded_S = pad_sequences(S, padding='post')

        return padded_A, padded_S
    
    def run(self, steps = 'all'):
        if steps == 'all':
            steps = ['load_data', 'create_action_state_pair', 'preprocess_text',
                     'prepare_action_state', 'w2v_model', 'get_embeddings']
        if isinstance(steps, list):
            
            # prepare data
            if 'load_data' in steps:
                df = self.load_data()
            if 'create_action_state_pair' in steps:
                df = self.create_action_state_pair(df)
                if 'create_action_state_pair' == steps[-1]:
                    return df
            if 'preprocess_text' in steps:
                df = self.preprocess_text(df)
                if 'preprocess_text' == steps[-1]:
                    return df
            
            # prepare model
            if 'prepare_action_state' in steps:
                action_sentences = list(df['judge_action_cleaned'].values)
                state_sentences = list(df['agent_state_cleaned'].values)
                if 'prepare_action_state' == steps[-1]:
                    return action_sentences, state_sentences, df
            if 'w2v_model' in steps:
                model = self.train_w2v(action_sentences, state_sentences)
                if 'w2v_model' == steps[-1]:
                    return model
            
            # get embeddings
            if 'get_embeddings' in steps:
                A, S = self.create_x_y(action_sentences, state_sentences, model)
                print("Action data shape:", A.shape, "\nState data shape:", S.shape) # n_samples, n_timesteps (in this case word sequence in a sent), n_features
                return A, S
        
        

## Save action_state_text to csv

In [29]:
my_preparer = PrepareData(input_directory = 'parsed_data', output_directory = 'corrected_action_state_pair', 
                          model_name = 'model_17to20_v3', min_duration=2, feature_size=300, 
                          context_window_size=5, min_words=1, down_sample=0.001)
action_sentences, state_sentences, df = my_preparer.run(steps = ['load_data', 'create_action_state_pair', 
                                                             'preprocess_text', 'prepare_action_state'])


Step: Loading data...
Input dataframe shape: (54108, 176)

Step: Creating action state pair...

Step: Preprocessing text...


In [31]:
df.to_csv('action_state_text_v4.csv')

{'John G. Roberts, Jr.': 0,
 'Clarence Thomas': 1,
 'Sonia Sotomayor': 2,
 'Stephen G. Breyer': 3,
 'Samuel A. Alito, Jr.': 4,
 'Neil Gorsuch': 5,
 'Elena Kagan': 6,
 'Brett M. Kavanaugh': 7,
 'Amy Coney Barrett': 8}

In [15]:
action_state_text = pd.concat([pd.Series([' '.join(sublist) for sublist in action_sentences], name='action'), 
pd.Series([' '.join(sublist) for sublist in state_sentences], name='state')], axis=1)
action_state_text.to_csv('action_state_text_v3.csv')

In [8]:
my_preparer = PrepareData(input_directory = 'parsed_data', output_directory = 'word2vec_model_apr28', 
                          model_name = 'model_17to20', min_duration=5, feature_size=300, 
                          context_window_size=5, min_words=1, down_sample=0.001)
action_sentences, state_sentences = my_preparer.run(steps = ['load_data', 'create_action_state_pair', 
                                                             'preprocess_text', 'prepare_action_state'])


Step: Loading data...
Input dataframe shape: (54108, 176)

Step: Creating action state pair...

Step: Preprocessing text...


In [21]:
action_state_text = pd.concat([pd.Series([' '.join(sublist) for sublist in action_sentences], name='action'), 
pd.Series([' '.join(sublist) for sublist in state_sentences], name='state')], axis=1)
action_state_text.to_csv('action_state_text.csv')

## Save the w2v embedding to pkl and action-state pairs to npz

In [7]:
my_preparer = PrepareData(input_directory = 'parsed_data', output_directory = 'word2vec_models', model_name = 'model_17to20',
               min_duration=5, feature_size=300, context_window_size=5, min_words=1, down_sample=0.001)

In [8]:
action, state = my_preparer.run()


Step: Loading data...
Input dataframe shape: (54108, 176)

Step: Creating action state pair...

Step: Preprocessing text...

Step: Training Word2Vec...
Action and state length: 11520

Step: Creating input X and y...
Action data shape: (11520, 217, 300) 
State data shape: (11520, 577, 300)


In [9]:
model = my_preparer.load_w2v()

Model vocabulary: 17569


In [11]:
from numpy import savez_compressed, load
if not os.path.exists('meta_learning'):
    os.mkdir('meta_learning')
savez_compressed('meta_learning/action.npz', action)

In [12]:
savez_compressed('meta_learning/state.npz', state)

In [14]:
a = load('meta_learning/action.npz')
a['arr_0'].shape

(11520, 217, 300)

In [ ]:
# knowledge base:
# judgment & jstor law reviews & open source examples & statements of laws & final judgment